In [2]:
import sknetwork as skn
from sknetwork.clustering import Louvain, get_modularity
from sknetwork.linalg import normalize
from sknetwork.utils import get_membership
from sknetwork.visualization import visualize_graph
import networkx as nx  
import torch
from torch_geometric.utils import to_scipy_sparse_matrix, to_networkx
from torch_geometric.data import Data
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from IPython.display import SVG

In [3]:
graph_dir = os.path.join('..','..', 'src', 'data', 'graph_data.pt')
graph = torch.load(graph_dir, weights_only=False)
data = graph['data']

# Extracting adjacency matrix from graph torch file
adjacency_matrix = to_scipy_sparse_matrix(data.edge_index)

# Extracting nodes position
G_nx = to_networkx(data, to_undirected=False)
pos = nx.spring_layout(G_nx)
positions = np.array([pos[i] for i in range(len(pos))])

# Clusterizing the graph
louvain = Louvain()
labels = louvain.fit_predict(adjacency_matrix)
labels_unique, counts = np.unique(labels, return_counts=True)
print(labels_unique, counts)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23] [1111  922  760  480  320  217  160  140  103    6    4    3    2    2
    2    2    2    2    2    2    2    2    2    2]


In [4]:
get_modularity(adjacency_matrix, labels)

np.float64(0.5210337850402389)

In [5]:
adjacency_aggregate = louvain.aggregate_
avg = normalize(get_membership(labels).T)
position_aggregate = avg.dot(positions)
labels_unique, counts = np.unique(labels, return_counts=True)

In [6]:
unique_clusters = np.unique(labels)

cluster_nodes = {}
for c in unique_clusters:
    cluster = np.where(labels == c)[0]
    if len(cluster) > 5: # If there are at least 10 users in the same cluser, preserves it
        cluster_nodes[c] = cluster

In [7]:
# Clusterize community in cluster i
mapping = graph['idx2user']
exp_clusters = {}

for k, cluster in cluster_nodes.items():
    users = []
    for node_id in cluster:
        username = mapping.get(node_id)
        users.append(username)

    exp_clusters[k] = users

In [8]:
def build_subgraph(cluster_id, users, edge_df, save_path):
    print(f"Processing cluster : {cluster_id} with {len(users)} users.")
    users = set(users)
    edges = edge_df[edge_df['source'].isin(users) & edge_df['target'].isin(users)]

    if len(edges) == 0:
        return

    # Mapping id - users 
    user2idx = {uid: idx for idx, uid in enumerate(users)}
    idx2user = {idx: uid for uid, idx in user2idx.items()}

    # Creating edges and associated weights
    edges_index = torch.tensor([
        [user2idx[src] for src in edges['source']],
        [user2idx[dst] for dst in edges['target']],
    ], dtype=torch.long)

    edges_weight = torch.tensor(edges['weight'].values, dtype=torch.float)

    # Creating the subgraph aka the k-th cluster
    x = torch.eye(len(user2idx))
    data = Data(x=x, edge_index=edges_index, edge_weight=edges_weight)
    torch.save({
        'data': data,
        'idx2user': idx2user,
    }, save_path)

    return save_path

In [ ]:
graph_dir = os.path.join('..','..', 'src', 'data', 'graph_data.pt')

def get_cluster_with_Louvain(subgraph_dir, threshold=0.55, min_users=5):
    subgraph = torch.load(subgraph_dir, weights_only=False)
    data = subgraph['data']

    # Extracting adjacency matrix of the subgraph
    adjacency_matrix = to_scipy_sparse_matrix(data.edge_index)

    louvain = Louvain()
    labels = louvain.fit_predict(adjacency_matrix)

    modularity = get_modularity(adjacency_matrix, labels)

    # This means that nodes are cohesive so there's no need to cluterize the subgraph more
    if modularity > threshold:
        return None
    
    print(f"Modularity: {modularity}")
    
    unique_clusters = np.unique(labels)
    cluster_nodes = {}

    for c in unique_clusters:
        cluster = np.where(labels == c)[0]
        if len(cluster) > min_users: # If there are at least 5 users in the same cluser, preserves it
            cluster_nodes[c] = cluster

    return cluster_nodes

In [31]:
def recursive_clustering(edge_df, subgraph_path, base_cluster_id, base_output_dir, cluster_tree, depth=0, max_depth = 4):
    # Stopping cluster creation at max_depth
    if depth >= max_depth:
        return
    
    new_clusters = get_cluster_with_Louvain(subgraph_path)
    if new_clusters is None:
        return
    
    subgraph = torch.load(subgraph_path, weights_only=False)
    idx2user = subgraph['idx2user']

    cluster_tree[int(base_cluster_id)] = {
        "users" : {str(idx2user[i]) for i in range(len(idx2user))},
        "children" : {}
    }

    child_tree = cluster_tree[int(base_cluster_id)]["children"]

    for sub_id, node_ids in new_clusters.items():
        users = [idx2user[idx] for idx in node_ids]

        cluster_id = f"{base_cluster_id}_{sub_id}"
        save_path = os.path.join(base_output_dir, f"subgraph_{cluster_id}.pt")
        
        new_path = build_subgraph(cluster_id, users, edge_df, save_path=save_path)
        if new_path:
            recursive_clustering(
            edge_df=edge_df,
            subgraph_path=new_path,
            base_cluster_id=cluster_id,
            base_output_dir=base_output_dir,
            cluster_tree = child_tree,
            depth=depth + 1,
            max_depth=max_depth
        )

In [32]:
# Wrapper to start the recursion
def start_recursive_clustering(first_clusters, edges_path, base_output_dir, max_depth=4):
    edges = pd.read_csv(edges_path)
    cluster_tree = {}

    for k, users in first_clusters.items():
        save_path = os.path.join(base_output_dir, f'subgraph_{k}.pt')
        new_cluster_dir = build_subgraph(k, users, edges, save_path)
        if new_cluster_dir is not None:
            recursive_clustering(
                subgraph_path=new_cluster_dir,
                base_cluster_id=k,
                edge_df=edges,
                base_output_dir=base_output_dir,
                cluster_tree=cluster_tree,
                depth=0,
                max_depth=max_depth
            )

    return cluster_tree

In [ ]:
import json

def make_json_serializable(obj):
    if isinstance(obj, dict):
        return {str(k): make_json_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, set):
        return list(obj)
    elif isinstance(obj, list):
        return [make_json_serializable(v) for v in obj]
    else:
        return obj

edges_path = os.path.join('..', '..', 'src', 'data', 'edges.csv')
save_dir = os.path.join('..', '..', 'src', 'graph_dir', 'subgraph_dir')
os.makedirs(save_dir, exist_ok=True)

cluster_tree = start_recursive_clustering(exp_clusters, edges_path, save_dir)
json_safe_tree = make_json_serializable(cluster_tree)

with open(os.path.join(save_dir, 'cluster_tree.json'), 'w', encoding='utf-8') as f:
    json.dump(json_safe_tree, f, indent=4, ensure_ascii=False)




Processing cluster : 0 with 1111 users.
Modularity: 0.4006980769906756
Processing cluster : 0_0 with 103 users.
Processing cluster : 0_1 with 101 users.
Modularity: 0.5484304399524376
Processing cluster : 0_1_0 with 19 users.
Modularity: 0.414039262343843
Processing cluster : 0_1_0_0 with 6 users.
Modularity: 0.24
Processing cluster : 0_1_1 with 13 users.
Modularity: 0.3829344432882414
Processing cluster : 0_1_2 with 12 users.
Modularity: 0.4291115311909263
Processing cluster : 0_1_3 with 12 users.
Modularity: 0.344
Processing cluster : 0_1_4 with 9 users.
Modularity: 0.4222222222222223
Processing cluster : 0_1_5 with 9 users.
Modularity: 0.3733333333333333
Processing cluster : 0_1_6 with 8 users.
Modularity: 0.3431952662721892
Processing cluster : 0_1_7 with 8 users.
Modularity: 0.35714285714285726
Processing cluster : 0_1_8 with 7 users.
Modularity: 0.31404958677685957
Processing cluster : 0_2 with 87 users.
Processing cluster : 0_3 with 87 users.
Processing cluster : 0_4 with 76 use